# ToDo:

## trancribe.py
- [ ] change transcriber to batch processing
- [ ] download data from s3 if not available, and delete afterwards
- [ ] save also timestamp

## generate_slurm.py
- [ ] create jobs based on the size of the data

In [9]:
import os

import torch
from transformers import AutoProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

import time

In [2]:
# cwd 
print(os.getcwd())

/Users/supriyatno/codes/transcribe_whisper


In [6]:
audio_dir = "world/snapshot.20240606153519/audio_files"

# list mp4 files
audio_files = [f for f in os.listdir(audio_dir) if f.endswith('.mp4')]

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [7]:
audio_batch = audio_files[:4]

In [10]:

processor = AutoProcessor.from_pretrained("openai/whisper-large-v3")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v3", torch_dtype=torch.float16)
model.to(device)

# rertieve 8 long audio sequences
raw_audio = [whisper.load_audio(audio_file) for audio_file in audio_batch]


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Error while downloading from https://cdn-lfs-us-1.huggingface.co/repos/75/9f/759fc969b3b76eb86aaf241244bcc4172d9fc1cf017a3595b0be65895ff7fa89/a8e94b85976e5864ba3e9525c7e6c83b2a1eca42d4b797a0c7c24d778e40fd95?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1719560194&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxOTU2MDE5NH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzc1LzlmLzc1OWZjOTY5YjNiNzZlYjg2YWFmMjQxMjQ0YmNjNDE3MmQ5ZmMxY2YwMTdhMzU5NWIwYmU2NTg5NWZmN2ZhODkvYThlOTRiODU5NzZlNTg2NGJhM2U5NTI1YzdlNmM4M2IyYTFlY2E0MmQ0Yjc5N2EwYzdjMjRkNzc4ZTQwZmQ5NT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=L5juzE2IgfnAFlQ1KIncKJk8OQbWWzeGWUZAlYsMw2IdnIGM-CPJBtqssl%7E3Wn4phh6vifLbM3oofNdJen3pXBswFjBiFlrXqbk9NVsIsh3kPyUbLdBl6eD

KeyboardInterrupt: 

In [ ]:
start_time = time.time()

# process input, make sure to pass `padding='longest'` and `return_attention_mask=True`
inputs = processor(raw_audio, return_tensors="pt", truncation=False, padding="longest", return_attention_mask=True, sampling_rate=16_000)
inputs = inputs.to("cuda", torch.float16)

# activate `temperature_fallback` and repetition detection filters and condition on prev text
result = model.generate(**inputs, condition_on_prev_tokens=False, temperature=(0.0, 0.2, 0.4, 0.6, 0.8, 1.0), logprob_threshold=-1.0, compression_ratio_threshold=1.35, return_timestamps=True)

decoded = processor.batch_decode(result, skip_special_tokens=True)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"{elapsed_time}")

print(decoded)

In [11]:
start_time = time.time()

with torch.no_grad():
    transcription = pipe(os.path.join(audio_dir, mp4_files[0]))

print(f"Time taken: {time.time() - start_time:.2f} seconds")    

Time taken: 29.00 seconds


In [8]:
mp4_files[0]

'v=_bhrRP5SElA.mp4'

In [9]:
csv_file = "world/snapshot.20240606153519/video_list_eng_title.csv"

In [10]:
import pandas as pd

file_df = pd.read_csv(csv_file)

In [13]:
file_df.head()

,Unnamed: 0,institution_id,institution_name,institution_alias,institution_established,institution_country,institution_city,channel_id,channel_title,channel_description,channel_url,video_id,video_title,video_description,video_published_at,video_duration,video_view_count,video_url,video_duration_seconds,is_english
0,0,https://ror.org/01kpzv902,Flinders University,Flinders University of South Australia,1966.0,Australia,Adelaide,UCykVMY_tZbXuZfCQTEYyvPg,Flinders University,Flinders is a leading international university...,https://youtube.com/channel/UCykVMY_tZbXuZfCQT...,KxQvUMYep4Y,Diploma in Sport Business with Adelaide United...,The Diploma in Sport Business is a one-year co...,5 months ago,23:08,73 views,https://www.youtube.com/watch?v=KxQvUMYep4Y,1388.0,True
1,1,https://ror.org/01kpzv902,Flinders University,Flinders University of South Australia,1966.0,Australia,Adelaide,UCykVMY_tZbXuZfCQTEYyvPg,Flinders University,Flinders is a leading international university...,https://youtube.com/channel/UCykVMY_tZbXuZfCQT...,4tNjwmxpuLM,2023 Chalmers Oration,The College of Medicine and Public Health host...,8 months ago,1:33:26,216 views,https://www.youtube.com/watch?v=4tNjwmxpuLM,5606.0,True
2,2,https://ror.org/01kpzv902,Flinders University,Flinders University of South Australia,1966.0,Australia,Adelaide,UCykVMY_tZbXuZfCQTEYyvPg,Flinders University,Flinders is a leading international university...,https://youtube.com/channel/UCykVMY_tZbXuZfCQT...,W-VhxLXIP2U,Fearless Conversations: Voice to Parliament,The Voice to Parliament Lecture focuses on the...,9 months ago,1:18:17,510 views,https://www.youtube.com/watch?v=W-VhxLXIP2U,4697.0,True
3,3,https://ror.org/01kpzv902,Flinders University,Flinders University of South Australia,1966.0,Australia,Adelaide,UCykVMY_tZbXuZfCQTEYyvPg,Flinders University,Flinders is a leading international university...,https://youtube.com/channel/UCykVMY_tZbXuZfCQT...,qGtXc8LHgVQ,Diploma in Sports Management - Port Adelaide A...,Hear about what it is like to study a Diploma ...,1 year ago,42:36,375 views,https://www.youtube.com/watch?v=qGtXc8LHgVQ,2556.0,True
4,4,https://ror.org/01kpzv902,Flinders University,Flinders University of South Australia,1966.0,Australia,Adelaide,UCykVMY_tZbXuZfCQTEYyvPg,Flinders University,Flinders is a leading international university...,https://youtube.com/channel/UCykVMY_tZbXuZfCQT...,3xqILHLwZxs,Fearless Conversations | Authenticity & Identi...,Watch our very special final Fearless Conversa...,1 year ago,51:08,114 views,https://www.youtube.com/watch?v=3xqILHLwZxs,3068.0,True


In [14]:
# sample 100 and save
file_df.sample(100).to_csv("world/snapshot.20240606153519/sample_100.csv", index=False)

In [8]:
from s3utils import get_list_of_files_s3, load_file_from_s3

import pandas as pd

In [2]:
files = get_list_of_files_s3("world/snapshot.20240606153519/audio_files")

In [9]:
# make a pandas datefram and sort
filenames, sizes = files

file_df = pd.DataFrame({"filename": filenames, "size": sizes})
file_df = file_df.sort_values("size", ascending=True)

In [10]:
file_df.head()

,filename,size
159153,world/snapshot.20240606153519/audio_files/oDUE...,665828
129902,world/snapshot.20240606153519/audio_files/ehDL...,1171307
99086,world/snapshot.20240606153519/audio_files/VVrT...,1301390
127501,world/snapshot.20240606153519/audio_files/dtka...,1583073
58853,world/snapshot.20240606153519/audio_files/IQpn...,2097152


In [13]:
# download the first 100 files
import os
for i, row in file_df.head(100).iterrows():
    filename = row["filename"]
    
    # check if dir not exists
    if not os.path.exists(os.path.dirname(filename)):
        os.makedirs(os.path.dirname(filename))
    
    print(f"Downloading {filename}")
    load_file_from_s3(filename)